<a href="https://colab.research.google.com/github/algo21-221040114/PytorchTest/blob/main/PytorchLightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytorch_lightning

In [ ]:
pip install torchmetrics

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split

from torchmetrics.functional import accuracy

import pytorch_lightning
from pytorch_lightning import LightningModule, Trainer

In [33]:
# Pytorch Lightning
# reference: https://zhuanlan.zhihu.com/p/410394574
class PLModel(LightningModule):
  
  def __init__(self, channels, height, width, hidden_size, batch_size):
    super().__init__()

    self.channels = channels
    self.height = height
    self.width = width
    self.hidden_size = hidden_size
    self.batch_size = batch_size

    self.mdoel = nn.Sequential(
        nn.Linear(self.channels * self.height * self.width, self.hidden_size*2),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(self.hidden_size*2, self.hidden_size),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(self.hidden_size, 1)
    )

  def forward(self, x):
    x = self.model(x)
    return x

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
    return optimizer

  def training_step(self, batch, batch_idx):
    x, y = batch, batch_idx
    pred_y = self.model(x)
    loss = nn.MSELoss(pred_y, y)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch, batch_idx
    pred_y = self.model(x)
    loss = nn.MSELoss(pred_y, y)

    self.log('val_loss', loss, prog_bar=True)
    return loss
    
  def test_step(self, batch, batch_idx):
      return self.validation_step(batch, batch_idx)

  def train_dataloader(self):
      return DataLoader(self.mnist_train, batch_size=BATCH_SIZE)

  def val_dataloader(self):
      return DataLoader(self.mnist_val, batch_size=BATCH_SIZE)

  def test_dataloader(self):
      return DataLoader(self.mnist_test, batch_size=BATCH_SIZE)

In [34]:
model = PLModel(channels=3, height=16, width=16, hidden_size=16, batch_size=8)

In [ ]:
AVAIL_GPUS = min(1, torch.cuda.device_count())

In [35]:
trainer = Trainer(
    accelerator="auto", # 使用accelerator来选择将要训练的硬件，可设置cpu、gpu、tpu以及自动选择
    max_epochs=50)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [30]:
train_data = torch.rand((16, 3, 16, 16))
train_label = torch.rand((16, 1))

In [ ]:
trainer.fit(model)